In [25]:
import tensorflow as tf

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


2.16.1
Num GPUs Available:  1


In [27]:
import pandas as pd
import json
# Load the datasets
train_data_path = './train.json' # Update this path
test_data_path = './test.json'   # Update this path

# Load the JSON data from the file
with open(train_data_path, 'r') as file:
    data = json.load(file)
    # Convert the loaded JSON data into a pandas DataFrame
    train_df = pd.DataFrame(data)
with open(test_data_path, 'r') as file:
    data = json.load(file)
    # Convert the loaded JSON data into a pandas DataFrame
    test_df = pd.DataFrame(data)
# Quick look at the data structure
print(train_df.head())
print(test_df.head())


   document                                          full_text  \
0         7  Design Thinking for innovation reflexion-Avril...   
1        10  Diego Estrada\n\nDesign Thinking Assignment\n\...   
2        16  Reporting process\n\nby Gilberto Gamboa\n\nCha...   
3        20  Design Thinking for Innovation\n\nSindy Samaca...   
4        56  Assignment:  Visualization Reflection  Submitt...   

                                              tokens  \
0  [Design, Thinking, for, innovation, reflexion,...   
1  [Diego, Estrada, \n\n, Design, Thinking, Assig...   
2  [Reporting, process, \n\n, by, Gilberto, Gambo...   
3  [Design, Thinking, for, Innovation, \n\n, Sind...   
4  [Assignment, :,   , Visualization,  , Reflecti...   

                                 trailing_whitespace  \
0  [True, True, True, True, False, False, True, F...   
1  [True, False, False, True, True, False, False,...   
2  [True, False, False, True, True, False, False,...   
3  [True, True, True, False, False, True, 

In [29]:
pii_types = [label for sublist in train_df['labels'].tolist() for label in sublist]
pii_types_df = pd.DataFrame(pii_types, columns=['PII_Type'])

# View the distribution of PII types
print(pii_types_df['PII_Type'].value_counts())

PII_Type
O                   4989794
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         20
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: count, dtype: int64


In [42]:
from transformers import BertTokenizerFast

# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Encode the text
original_text = train_df['full_text'].iloc[0]
original_tokens = train_df['tokens'].iloc[0]
original_labels = train_df['labels'].iloc[0]

full_text = original_text#"innovation reflexion-Avril 2021-Nathalie Sylla\n\n"
tokens = original_tokens#["innovation","reflexion","-","Avril","2021","-","Nathalie","Sylla","\n\n"]
labels = original_labels#["O","O","O","O","O","O","B-NAME_STUDENT","I-NAME_STUDENT","O"]

encoded = tokenizer.encode_plus(full_text, return_offsets_mapping=True, padding='max_length', truncation=True)
new_tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'])
# Print the result
print(new_tokens)
print(len(encoded['input_ids']))
print(encoded['input_ids'])
print(encoded['offset_mapping'])


['[CLS]', 'design', 'thinking', 'for', 'innovation', 'reflex', '##ion', '-', 'av', '##ril', '2021', '-', 'nat', '##hal', '##ie', 'sy', '##lla', 'challenge', '&', 'selection', 'the', 'tool', 'i', 'use', 'to', 'help', 'all', 'stakeholders', 'finding', 'their', 'way', 'through', 'the', 'complexity', 'of', 'a', 'project', 'is', 'the', 'mind', 'map', '.', 'what', 'exactly', 'is', 'a', 'mind', 'map', '?', 'according', 'to', 'the', 'definition', 'of', 'bu', '##zan', 't', '.', 'and', 'bu', '##zan', 'b', '.', '(', '1999', ',', 'des', '##sin', '##e', '-', 'moi', 'l', "'", 'intelligence', '.', 'paris', ':', 'les', 'editions', 'd', "'", 'organisation', '.', ')', ',', 'the', 'mind', 'map', '(', 'or', 'he', '##uri', '##stic', 'diagram', ')', 'is', 'a', 'graphic', 'representation', 'technique', 'that', 'follows', 'the', 'natural', 'functioning', 'of', 'the', 'mind', 'and', 'allows', 'the', 'brain', "'", 's', 'potential', 'to', 'be', 'released', '.', 'cf', 'annex', '##1', 'this', 'tool', 'has', 'many'

In [44]:
def segment_text_by_tokens(original_tokens, original_labels, max_length, overlap):
    # Adjust max_length to account for special tokens like [CLS] and [SEP] which are added by models like BERT
    max_length -= 2  # Assuming 2 special tokens: [CLS] and [SEP]

    segments = []
    current_tokens = []
    current_labels = []

    for token, label in zip(original_tokens, original_labels):
        # Check if adding the next token would exceed the max_length
        if len(current_tokens) >= max_length and current_tokens:
            # Finalize the current segment before the overflow
            segments.append({
                "tokens": current_tokens,
                "labels": current_labels
            })
            # Start new segment with overlap if specified
            current_tokens = current_tokens[-overlap:] if overlap else []
            current_labels = current_labels[-overlap:] if overlap else []

        current_tokens.append(token)
        current_labels.append(label)
    
    # Add the last segment if there are remaining tokens
    if current_tokens:
        segments.append({
            "tokens": current_tokens,
            "labels": current_labels
        })
    
    return segments

# Example usage
# original_text = "innovation reflexion-Avril 2021-Nathalie Sylla\n\n"
# original_tokens = ["innovation","reflexion","-","Avril","2021","-","Nathalie","Sylla","\n\n"]
# original_labels = ["O","O","O","O","O","O","B-NAME_STUDENT","I-NAME_STUDENT","O"]
max_length = 512  # Maximum token count, including special tokens like [CLS] and [SEP]
overlap = 16  # Example overlap

segments = segment_text_by_tokens(original_tokens, original_labels, max_length, overlap)
for i, segment in enumerate(segments):
    print(f"Segment {i+1}:")
    # print(f"Text: {segment['text']}")
    print(f"Tokens: {segment['tokens']}")
    print(f"Labels: {segment['labels']}\n")


Segment 1:
Tokens: ['Design', 'Thinking', 'for', 'innovation', 'reflexion', '-', 'Avril', '2021', '-', 'Nathalie', 'Sylla', '\n\n', 'Challenge', '&', 'selection', '\n\n', 'The', 'tool', 'I', 'use', 'to', 'help', 'all', 'stakeholders', 'finding', 'their', 'way', 'through', 'the', 'complexity', 'of', 'a', 'project', 'is', 'the', ' ', 'mind', 'map', '.', '\n\n', 'What', 'exactly', 'is', 'a', 'mind', 'map', '?', 'According', 'to', 'the', 'definition', 'of', 'Buzan', 'T.', 'and', 'Buzan', 'B.', '(', '1999', ',', 'Dessine', '-', 'moi', ' ', "l'intelligence", '.', 'Paris', ':', 'Les', 'Éditions', "d'Organisation", '.', ')', ',', 'the', 'mind', 'map', '(', 'or', 'heuristic', 'diagram', ')', 'is', 'a', 'graphic', ' ', 'representation', 'technique', 'that', 'follows', 'the', 'natural', 'functioning', 'of', 'the', 'mind', 'and', 'allows', 'the', 'brain', "'s", ' ', 'potential', 'to', 'be', 'released', '.', 'Cf', 'Annex1', '\n\n', 'This', 'tool', 'has', 'many', 'advantages', ':', '\n\n', '•', ' ',

In [33]:
# Step 1: Generate Original Token Offset Mapping
original_token_offsets = []
cursor = 0
for token in tokens:
    start = full_text.find(token, cursor)
    end = start + len(token)
    cursor = end
    original_token_offsets.append((start, end))

# Step 2: Tokenize and Align Labels
encoded = tokenizer(full_text, return_offsets_mapping=True, truncation=True, padding=True)
new_tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'])
new_labels = ['O'] * len(new_tokens)  # Initialize all labels as 'O'

for i, offset in enumerate(encoded['offset_mapping']):
    if offset == (0, 0):  # Skip special tokens
        continue
    # Find the original token that encompasses the new token
    for j, (orig_start, orig_end) in enumerate(original_token_offsets):
        # Check if the new token is within the span of the original token
        if orig_start <= offset[0] and offset[1] <= orig_end:
            # Assign the original token's label to the new token
            if orig_start == offset[0]:
                new_labels[i] = labels[j]
            else:
                new_labels[i] = labels[j].replace('B-', 'I-')
            break

# Handling subwords and continuation labels: Adjust the new_labels to use continuation labels ('I-') where necessary
for i in range(1, len(new_labels)):
    if new_labels[i].startswith('B-') and new_labels[i-1] == new_labels[i].replace('B-', 'I-'):
        new_labels[i] = new_labels[i].replace('B-', 'I-')

# Step 3: Review the results
print(new_tokens)
print(new_labels)


['[CLS]', 'design', 'thinking', 'for', 'innovation', 'reflex', '##ion', '-', 'av', '##ril', '2021', '-', 'nat', '##hal', '##ie', 'sy', '##lla', 'challenge', '&', 'selection', 'the', 'tool', 'i', 'use', 'to', 'help', 'all', 'stakeholders', 'finding', 'their', 'way', 'through', 'the', 'complexity', 'of', 'a', 'project', 'is', 'the', 'mind', 'map', '.', 'what', 'exactly', 'is', 'a', 'mind', 'map', '?', 'according', 'to', 'the', 'definition', 'of', 'bu', '##zan', 't', '.', 'and', 'bu', '##zan', 'b', '.', '(', '1999', ',', 'des', '##sin', '##e', '-', 'moi', 'l', "'", 'intelligence', '.', 'paris', ':', 'les', 'editions', 'd', "'", 'organisation', '.', ')', ',', 'the', 'mind', 'map', '(', 'or', 'he', '##uri', '##stic', 'diagram', ')', 'is', 'a', 'graphic', 'representation', 'technique', 'that', 'follows', 'the', 'natural', 'functioning', 'of', 'the', 'mind', 'and', 'allows', 'the', 'brain', "'", 's', 'potential', 'to', 'be', 'released', '.', 'cf', 'annex', '##1', 'this', 'tool', 'has', 'many'